In [78]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from pyslfp import FingerPrint, plot_SHGrid, ResponseFields
import numpy as np

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
# Set some things up.
def inner_product(fp, f1, f2):
    g = fp.gravitational_acceleration
    b = fp.mean_sea_floor_radius
    return fp.integrate(f1.sl * f2.sl)  \
        - (1 /g) * fp.integrate((f1.u * f2.u + f1.phi * f2.phi))  \
        - (1 /g) * np.inner(f1.omega, f2.omega)

def inner_product2(fp, f1, f2):
    g = fp.gravitational_acceleration
    return fp.integrate(f1.sl * f2.sl + f1.u * f2.u + f1.phi * f2.phi) + np.inner(f1.omega, f2.omega)

# Set up the fingerprint solver. 
finger_print = FingerPrint(128)

# Set the background sea level and ice thickness. 
finger_print.set_background_state_from_ice_ng()

# Set the load. 
zeta1 = finger_print.northern_hemisphere_load()
zeta2 = finger_print.southern_hemisphere_load()
zero = zeta1 - zeta1

In [103]:
# Invent a generalised load
generalised_load1 = ResponseFields(zeta1, zeta1, np.array([1e25,1e25]), zeta1)
generalised_load2 = ResponseFields(zeta2, zeta2, np.array([5e25,3e25]), zeta2)

# Compute the response. 
response1 = finger_print.generalised_solver(generalised_load1, verbose=True)
response2 = finger_print.generalised_solver(generalised_load2)

# Needs to be a full inner product (i.e. not just zeta*sl)
print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

# Plot the sea level change over the oceans. 
#plot_SHGrid(response1.sl * finger_print.ocean_mask() , contour=True, levels=60, colorbar=True)

Iteration = 1, relative error = 4.1622e-02
Iteration = 2, relative error = 1.9158e-03
Iteration = 3, relative error = 2.2750e-04
Iteration = 4, relative error = 3.6457e-05
Iteration = 5, relative error = 6.6913e-06
Iteration = 6, relative error = 1.2998e-06
Iteration = 7, relative error = 2.5963e-07
-7.736755404912344e+18
3.6266609514180854e+18


In [62]:
## Check to see if generalised solver is consistent with the normal solver.

generalised_load1 = ResponseFields(zero, zero, np.array([0,0]), zeta1)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.solver(zeta1)

print(finger_print.integrate(response1.sl * zeta1))
print(finger_print.integrate(response2.sl * zeta1))

2.6831278712261214e+20
2.6831278712261214e+20


In [64]:
## Try u only

generalised_load1 = ResponseFields(zeta1, zero, np.array([0,0]), zero)
generalised_load2 = ResponseFields(zeta2, zero, np.array([0,0]), zero)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

-1.5516739250868074e+19
-1.551673924583832e+19


In [65]:
## Try phi only

generalised_load1 = ResponseFields(zero, zeta1, np.array([0,0]), zero)
generalised_load2 = ResponseFields(zero, zeta2, np.array([0,0]), zero)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

5.950552193828723e+19
5.9505522057528435e+19


In [66]:
## Try zeta only

generalised_load1 = ResponseFields(zero, zero, np.array([0,0]), zeta1)
generalised_load2 = ResponseFields(zero, zero, np.array([0,0]), zeta2)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

-1.3615261520789651e+19
-1.361503064448356e+19


In [67]:
## Try k only

generalised_load1 = ResponseFields(zero, zero, np.array([1e25,2e25]), zero)
generalised_load2 = ResponseFields(zero, zero, np.array([5e25,3e25]), zero)
response1 = finger_print.generalised_solver(generalised_load1)
response2 = finger_print.generalised_solver(generalised_load2)

print(inner_product(finger_print, generalised_load1, response2))
print(inner_product(finger_print, generalised_load2, response1))

822412556825230.6
822412556825230.6
